In [3]:
import numpy as np 
import plotly.graph_objects as go
from scipy.optimize import minimize
import pandas as pd

In [34]:
file_path = 'data/returns.csv'
rf_rate = 0.0653

In [35]:
try:
    all_returns = pd.read_csv(file_path, index_col=0, parse_dates=True)
except FileNotFoundError:
    print(f"ERROR: The file '{file_path}' was not found.")
    print("Please make sure your 'returns.csv' file is inside a 'data' folder.")
    raise

In [36]:
tickers = all_returns.columns[:5].tolist()
stock_returns = all_returns[tickers]

In [37]:
stock_returns.head()

,OLAELEC,WHIRLPOOL,SAGILITY,RAMCOCEM,TATACHEM
date,,,,,
2025-10-10 00:00:00+05:30,-0.010010,0.021958,0.035114,0.010312,-0.003317
2025-10-09 00:00:00+05:30,-0.020310,-0.017588,-0.006013,0.009568,-0.002040
2025-10-08 00:00:00+05:30,-0.026581,-0.024600,0.006236,-0.007085,-0.015411
2025-10-07 00:00:00+05:30,-0.000190,0.008499,0.012816,0.026397,-0.005139
2025-10-06 00:00:00+05:30,-0.025703,0.003301,0.017118,-0.010410,0.001350


In [38]:
mean_returns = stock_returns.mean()
cov_matrix = stock_returns.cov()

In [39]:
def portf_stats(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns * weights) * 252
    std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return returns, std_dev

def neg_sharpe(weights, mean_returns, cov_matrix, rf_rate):
    returns, std_dev = portf_stats(weights, mean_returns, cov_matrix)
    sharpe_ratio = (returns-rf_rate)/std_dev
    return -sharpe_ratio

def portf_volatility(weights, mean_returns, cov_matrix):
    return portf_stats(weights, mean_returns, cov_matrix)[1]

In [40]:
def tangency_portfolio(mean_returns, cov_matrix, rf_rate):
    numAssets = len(mean_returns)
    args = (mean_returns, cov_matrix, rf_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for asset in range(numAssets))
    initial_guess = numAssets * [1. / numAssets]
    res = minimize(
        neg_sharpe,
        initial_guess,
        args,
        method = 'SLSQP',
        constraints=constraints,
        bounds=bounds
    )
    if not res.success:
        raise BaseException(res.message)
    return res

In [41]:
tangency_res = tangency_portfolio(mean_returns, cov_matrix, rf_rate)
tangency_weights = tangency_res.x
tangency_return, tangency_vol = portf_stats(tangency_weights, mean_returns, cov_matrix)
tangency_sharpe = (tangency_return - rf_rate) / tangency_vol

In [42]:
def min_var_portfolio(mean_returns, cov_matrix):
    numAssets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for asset in range(numAssets))
    initial_guess = numAssets * [1. / numAssets]
    res = minimize(
        portf_volatility,
        initial_guess,
        args=args,
        method = 'SLSQP',
        constraints=constraints,
        bounds=bounds
    )
    if not res.success:
        raise BaseException(res.message)
    return res

In [43]:
def efficient_portfolio(mean_returns, cov_matrix, target_returns):
    numAssets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type' : 'eq', 'fun': lambda x: portf_stats(x, mean_returns, cov_matrix)[0] - target_returns}, 
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for asset in range(numAssets))
    initial_guess = numAssets * [1. / numAssets]
    res = minimize(
        portf_volatility,
        initial_guess,
        args=args,
        method = 'SLSQP',
        constraints=constraints,
        bounds=bounds
    )
    return res

In [44]:
min_var_result = min_var_portfolio(mean_returns, cov_matrix)
min_var_weights = min_var_result.x
min_var_return, min_var_vol = portf_stats(min_var_weights, mean_returns, cov_matrix)

In [47]:
index_ticker = all_returns.columns[-1]
index_returns = all_returns[index_ticker]

In [49]:
def getBenchmarkPerformance(index_returns, rf_rate):
    meanReturn = index_returns.mean() * 252
    stdDev = index_returns.std() * np.sqrt(252)
    sharpe = (meanReturn - rf_rate) / stdDev
    return meanReturn, stdDev, sharpe